What I'm Trying top predict for this project is a number of rings on the shell of an Abalone ( A type of mollusk ) they are also called sea ears.
The number of rings is supposed to represent the age.

![Abalone](https://upload.wikimedia.org/wikipedia/commons/thumb/3/33/LivingAbalone.JPG/220px-LivingAbalone.JPG)

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_log_error

In [7]:
train_file = "./train.csv"
test_file = "./test.csv"

train = pd.read_csv(train_file, index_col=0)
test = pd.read_csv(test_file, index_col=0)
print(train.head())
print(test.head())

   Sex  Length  Diameter  Height  Whole weight  Whole weight.1  \
id                                                               
0    F   0.550     0.430   0.150        0.7715          0.3285   
1    F   0.630     0.490   0.145        1.1300          0.4580   
2    I   0.160     0.110   0.025        0.0210          0.0055   
3    M   0.595     0.475   0.150        0.9145          0.3755   
4    I   0.555     0.425   0.130        0.7820          0.3695   

    Whole weight.2  Shell weight  Rings  
id                                       
0           0.1465        0.2400     11  
1           0.2765        0.3200     11  
2           0.0030        0.0050      6  
3           0.2055        0.2500     10  
4           0.1600        0.1975      9  
      Sex  Length  Diameter  Height  Whole weight  Whole weight.1  \
id                                                                  
90615   M   0.645     0.475   0.155        1.2380          0.6185   
90616   M   0.580     0.460   0.160 

In [8]:
encoder = LabelEncoder()

train['Sex'] = encoder.fit_transform(train['Sex'])
test['Sex'] = encoder.transform(test['Sex'])
print(train['Sex'].head())

id
0    0
1    0
2    1
3    2
4    1
Name: Sex, dtype: int32


In [9]:
X = train.drop('Rings', axis=1)
y = train['Rings']

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [10]:
linreg = LinearRegression()

linreg.fit(X_train, y_train)

y_pred = np.round(linreg.predict(X_test))
y_pred[y_pred<1] = 1                          # Remove negative values and set them to 1

In [11]:
rmsle = root_mean_squared_log_error(y_true=y_test, y_pred=y_pred)
print(rmsle)

0.16745761106008042


That's a good baseline i think. would like to get it to at least 0.14 so i can get on the leaderboard. This will be my baseline.

### Submission 2

In [12]:
from sklearn.ensemble import RandomForestRegressor

In [14]:
RandomForestRegressor().fit(X_train, y_train)

RandomForestRegressor()

In [17]:
param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [4, 6, 8],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["auto", "sqrt", "log2"]  
}

rf = RandomForestRegressor()

grid_search = RandomizedSearchCV(rf, param_grid, cv=3, scoring='neg_mean_squared_error')

grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

print("Best Hyperparameters:", best_params)

C:\Users\vosni\anaconda3\envs\block_c_tf\lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
12 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\vosni\anaconda3\envs\block_c_tf\lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\vosni\anaconda3\envs\block_c_tf\lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "C:\Users\vosni\anaconda3\envs\block_c_tf\lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_

Best Hyperparameters: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_depth': 8}


In [18]:
rf_final_2 = RandomForestRegressor(**best_params)

rf_final_2.fit(X, y)

y_pred_final_2 = np.round(rf_final_2.predict(test))
y_pred_final_2[y_pred_final_2 < 1] = 1

C:\Users\vosni\anaconda3\envs\block_c_tf\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [19]:
df_final_2 = pd.DataFrame()
df_final_2.index = test.index
df_final_2['Rings'] = y_pred_final_2

print(df_final_2.head())

       Rings
id          
90615   11.0
90616   10.0
90617   10.0
90618   10.0
90619   10.0


In [20]:
df_final_2['Rings'] = df_final_2['Rings'].astype('int')
print(df_final_2.head())

       Rings
id          
90615     11
90616     10
90617     10
90618     10
90619     10


In [21]:
df_final_2.to_csv('submission_2.csv')